In [1]:
# Mount Google Drive (If using Colab)

from google.colab import drive
drive.mount("drive")

Mounted at drive


In [2]:
# Install Required Libraries from PyPI

!pip install face-detection
!pip install tqdm

  Created wheel for face-detection: filename=face_detection-0.2.1-cp36-none-any.whl size=21403 sha256=39d27d4f8b8336a5a17459784c3ff388720f24e2aa4ee9300744febf24bb6389
  Stored in directory: /root/.cache/pip/wheels/7d/45/54/d117d6cc260f31a22d15cda860ff46325af314cac687848ab7
Successfully built face-detection


In [3]:
# Import Required Libraries

import os
import numpy as np
import cv2
import face_detection
from keras.applications import resnet50
from keras.models import Model
from keras.layers import Dense,Dropout,AveragePooling2D,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tqdm
from google.colab.patches import cv2_imshow

In [4]:
# Initialize a Face Detector 
# Confidence Threshold can be Adjusted, Greater values would Detect only Clear Faces

detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

Downloading: "http://folk.ntnu.no/haakohu/WIDERFace_DSFD_RES152.pth" to /root/.cache/torch/hub/checkpoints/WIDERFace_DSFD_RES152.pth


## BLURRING AUGMENTATION ##

In [5]:
# Define Blurring Kernel Size Ranges, a Random Size would be chosen in the Specified Ranges
# Greater the Size, Higher is the Blurring Effect (Adjustments can be made according to the needs)

motion_blur_kernel_range = (4,8)  
average_blur_kernel_range = (3,7)
gaussian_blur_kernel_range = (3,8)

In [6]:
# Set Blurring Kernels to Use and their associated Probabilities

Blurring_Kernels = ["none","motion","gaussian","average"]
Probs = [0.75,0.1,0.05,0.1]

In [7]:
# Add Motion Blur to an Image in a Random Direction

def motion_blur(img):

  # Choose a Random Kernel Size
  kernel_size = np.random.randint(motion_blur_kernel_range[0],motion_blur_kernel_range[1])
  kernel = np.zeros((kernel_size, kernel_size))

  # Random Selection of Direction of Motion Blur
  types = ["vertical","horizontal","main_diagonal","anti_diagonal"]
  choice = np.random.choice(types)

  if choice=="vertical":
    kernel[:,int((kernel_size-1)/2)] = np.ones(kernel_size)/kernel_size
  
  elif choice=="horizontal":
    kernel[int((kernel_size-1)/2),:] = np.ones(kernel_size)/kernel_size
  
  elif choice=="main_diagonal":
    
    for i in range(kernel_size):
      kernel[i][i] = 1/kernel_size

  elif choice=="anti_diagonal":

    for i in range(kernel_size):
      kernel[i][kernel_size-i-1] = 1/kernel_size

  # Convolve and Return the Blurred Image
  return cv2.filter2D(img,-1,kernel)

In [8]:
# Add a Random Blur Effect to an Image with a Random Kernel Size (in the Specified Ranges)

def get_blurred_picture(img):
  
  # Randomly choose a Blurring Technique
  choice = np.random.choice(Blurring_Kernels,p=Probs)

  # RGB to BGR for OpenCV
  img = img[:,:,::-1]

  if choice=="none":

    random_blurred_img = img

  elif choice=="motion":
  
    random_blurred_img  = motion_blur(img)

  elif choice=="gaussian":

    kernel_size = np.random.randint(gaussian_blur_kernel_range[0],gaussian_blur_kernel_range[1])

    if kernel_size%2==0:
      kernel_size-=1
    
    random_blurred_img = cv2.GaussianBlur(img,(kernel_size,kernel_size),0)

  elif choice=="average":
  
    kernel_size = np.random.randint(average_blur_kernel_range[0],average_blur_kernel_range[1])
    random_blurred_img = cv2.blur(img,(kernel_size,kernel_size))

  # PreProcess for ResNet50
  preprocessed = resnet50.preprocess_input(random_blurred_img[:,:,::-1])
  
  return preprocessed 

## TRAINING ##

In [9]:
# Load Pretrained ResNet50 Model (without Last few Layers) 
# Freeze all the Layers

base_network  = resnet50.ResNet50(input_shape = (224,224,3),weights='imagenet',include_top=False)
for layer in base_network.layers:
  layer.trainaibale = False

94773248/94765736 [==============================] - 1s 0us/step


In [10]:
# Define the Face Mask Classifier Model by adding a few Layers on top of the ResNet50 Pretrained Model

classifier_network = base_network.output
classifier_network = AveragePooling2D(pool_size=(7, 7),name = "Average_Pool_Final")(classifier_network)
classifier_network = Flatten(name = "Flatten_Final")(classifier_network)
classifier_network = Dense(128, activation="relu",name = "Dense_Final")(classifier_network)
classifier_network = Dropout(0.5,name = "Dropout_Final")(classifier_network)
classifier_network = Dense(1, activation="sigmoid", name = "Sigmoid_Classifier" )(classifier_network)

mask_classifier = Model(inputs=base_network.input, outputs=classifier_network)

In [13]:
# Set Path to the Dataset
# Faces would be extracted and placed in the specified Directory after Processing

Dataset_PATH = "/content/drive/MyDrive/Social_Distancing_with_CV/Training_Data"
Processed_Dataset_PATH = "/content/drive/MyDrive/Social_Distancing_with_CV/Processed_Training_Data"

# Create Empty Directories
os.mkdir(Processed_Dataset_PATH)
os.mkdir(os.path.join(Processed_Dataset_PATH,"with_mask"))
os.mkdir(os.path.join(Processed_Dataset_PATH,"without_mask"))

In [14]:
# Prepare the Data for Training 
# Extract Faces from the Dataset and Save them in the specified Directory

# There should be 2 Sub-Directories corresponding to Masked and Non-Masked Faces
paths = ["with_mask","without_mask"]

for path in paths:

  curr_path = os.path.join(Dataset_PATH,path)
  
  # Loop through all Images
  for file_name in tqdm.notebook.tqdm(os.listdir(curr_path)):

    try:
      
      image = cv2.imread(os.path.join(curr_path,file_name))

      # Detect Faces, Crop and Save
      detections = detector.detect(image[:,:,::-1])

      for j in range(len(detections)):
    
        face = image[int(detections[j][1]):int(detections[j][3]),
                     int(detections[j][0]):int(detections[j][2])]
        
        cv2.imwrite(os.path.join(Processed_Dataset_PATH,path)+"/"+file_name,face)
    
    except:
      continue

In [23]:
# Set the Hyper-Parameters

alpha = 0.00001
n_epochs = 5
mini_batch_size = 16

In [24]:
# Compile the Model

opt = Adam(learning_rate=alpha,decay = alpha/n_epochs)
mask_classifier.compile(optimizer=opt,loss="binary_crossentropy",metrics = ['accuracy'])

In [25]:
# Define a ImageDataGenerator for Real-Time Data Augmentation
# Parameters can be Tuned for controlling the Augmentation

Data_Generator = ImageDataGenerator(horizontal_flip=True,
                                    brightness_range=[0.5,1.25],
                                    zoom_range=[0.8,1],
                                    rotation_range = 15,
                                    preprocessing_function=get_blurred_picture)

In [26]:
# Create a Data_Generator Instance

Train_Data_Generator = Data_Generator.flow_from_directory(Processed_Dataset_PATH,target_size=(224,224),
                                                          class_mode="binary",batch_size=mini_batch_size)

Found 1373 images belonging to 2 classes.


In [27]:
# Train the Model

mask_classifier.fit(x=Train_Data_Generator,
                    steps_per_epoch=(Train_Data_Generator.n//mini_batch_size),
                    epochs=n_epochs)

Epoch 1/5
85/85 [==============================] - 24s 277ms/step - loss: 0.4087 - accuracy: 0.8180
Epoch 2/5
85/85 [==============================] - 24s 281ms/step - loss: 0.1361 - accuracy: 0.9514
Epoch 3/5
85/85 [==============================] - 24s 280ms/step - loss: 0.0816 - accuracy: 0.9713
Epoch 4/5
85/85 [==============================] - 24s 280ms/step - loss: 0.0586 - accuracy: 0.9808
Epoch 5/5
85/85 [==============================] - 24s 281ms/step - loss: 0.0513 - accuracy: 0.9853


In [28]:
# Test the Model on a Picture

FILE_PATH = "drive/My Drive/Social_Distancing_with_AI/test.jpg"

img = cv2.imread(FILE_PATH)
masked_faces = []
unmasked_faces = []

# Detect Faces
detections = detector.detect(img[:,:,::-1])

if detections.shape[0]>0:
      
  for i in range(detections.shape[0]):
  
    # Get Co-ordinates
    x1 = int(detections[i][0])
    x2 = int(detections[i][2])
    y1 = int(detections[i][1])
    y2 = int(detections[i][3])

    # Predict Output
    face_arr = cv2.resize(img[y1:y2,x1:x2,::-1], (224, 224), interpolation=cv2.INTER_NEAREST)
    face_arr = np.expand_dims(face_arr, axis=0)
    face_arr = resnet50.preprocess_input(face_arr)
    match = mask_classifier.predict(face_arr)

    if match[0][0]<0.5:
      masked_faces.append([x1,y1,x2,y2])
    else:
      unmasked_faces.append([x1,y1,x2,y2])

# Put Bounding Box on the Faces (Green:Masked,Red:Not-Masked)
for f in range(len(masked_faces)):

  a,b,c,d = masked_faces[f]
  cv2.rectangle(img, (a,b), (c,d), (0,255,0), 2)

for f in range(len(unmasked_faces)):

  a,b,c,d = unmasked_faces[f]
  cv2.rectangle(img, (a,b), (c,d), (0,0,255), 2)

# Show Results
cv2_imshow(img)

In [ ]:
# Save the Trained Weights to Disk

SAVE_PATH = "drive/My Drive/Social_Distancing_with_AI/ResNet50_Classifier.h5"
mask_classifier.save(SAVE_PATH)